# Task 1 - Data Collection and Preparation
<i>You have been provided with a full dataset of charging sessions
in two sites for an extended timeframe. The first step should start by ingesting the data in appropriate
format, checking for missing or erroneous data, and cleaning your dataset for use in later stages of
your project. Briefly describe how you proceeded and how you dealt with possible missing/erroneous
data.</i>

#### 1.1 Cleaning Charging Sessions

In [5]:
# All imports are listed and executed here:
import numpy as np
import pandas as pd
import json
import pytz
from datetime import datetime
import pytz
import matplotlib.pyplot as plt
from ipywidgets import *
pd.options.mode.chained_assignment = None  # default='warn'

In [6]:
# Load charging data and have first look on it:
df_charging_sessions = pd.DataFrame(pd.read_csv("../datasets/charging_sessions.csv"))
df_charging_sessions.head(5)

,Unnamed: 0,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs
0,0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil..."
1,1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil..."
2,2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
3,3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1_1_193_820_2020-01-02 13:59:58.309319,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
4,4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1_1_193_819_2020-01-02 14:00:00.779967,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'mil..."


In [7]:
# Show information about all columns:
df_charging_sessions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66450 entries, 0 to 66449
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        66450 non-null  int64  
 1   id                66450 non-null  object 
 2   connectionTime    66450 non-null  object 
 3   disconnectTime    66450 non-null  object 
 4   doneChargingTime  62362 non-null  object 
 5   kWhDelivered      66450 non-null  float64
 6   sessionID         66450 non-null  object 
 7   siteID            66450 non-null  int64  
 8   spaceID           66450 non-null  object 
 9   stationID         66450 non-null  object 
 10  timezone          66450 non-null  object 
 11  userID            49187 non-null  float64
 12  userInputs        49187 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usage: 6.6+ MB


In [8]:
# Drop unneccessary id column and timezone column, whch is identical for all entries:
df_charging_sessions = df_charging_sessions.drop(["Unnamed: 0","timezone"], axis=1)
# Drop all duplicates and keep first duplicated entry:
df_charging_sessions = df_charging_sessions.drop_duplicates(keep="first")

In [9]:
# Check the kWh Delivered value for feasibility:
print("Minimum amount kWh delivered:", df_charging_sessions["kWhDelivered"].min(),"kWh")
print("Maximum amount kWh delivered:", df_charging_sessions["kWhDelivered"].max(),"kWh")
print("Average amount kWh delivered:", df_charging_sessions["kWhDelivered"].mean(),"kWh")

Minimum amount kWh delivered: 0.501 kWh
Maximum amount kWh delivered: 108.79724166666666 kWh
Average amount kWh delivered: 11.734288174559863 kWh


In [10]:
# Look at unique values of site/space/station ID's:
print("Unique Site ID's:", df_charging_sessions["siteID"].unique())
print("Unique Space ID's:", df_charging_sessions["spaceID"].unique())
print("Unique Station ID's:", df_charging_sessions["stationID"].unique())

Unique Site ID's: [1 2]
Unique Space ID's: ['AG-3F30' 'AG-1F01' 'AG-1F03' 'AG-1F04' 'AG-1F06' 'AG-1F13' 'AG-1F09'
 'AG-4F34' 'AG-1F11' 'AG-1F08' 'AG-3F22' 'AG-1F14' 'AG-1F07' 'AG-1F12'
 'AG-3F29' 'AG-1F05' 'AG-3F18' 'AG-1F10' 'AG-3F16' 'AG-4F37' 'AG-1F02'
 'AG-3F28' 'AG-3F25' 'AG-3F24' 'AG-3F33' 'AG-3F23' 'AG-3F31' 'AG-3F32'
 'AG-3F27' 'AG-3F26' 'AG-3F20' 'AG-3F17' 'AG-3F21' 'AG-3F19' 'AG-4F44'
 'AG-4F52' 'AG-4F41' 'AG-4F48' 'AG-4F35' 'AG-4F50' 'AG-3F15' 'AG-4F51'
 'AG-4F47' 'AG-4F46' 'AG-4F49' 'AG-4F43' 'AG-4F42' 'AG-4F45' 'AG-4F36'
 'AG-4F38' 'AG-4F40' 'AG-4F39' 'CA-512' 'CA-327' 'CA-326' 'CA-303'
 'CA-325' 'CA-311' 'CA-309' 'CA-491' 'CA-493' 'CA-499' 'CA-323' 'CA-313'
 'CA-324' 'CA-513' 'CA-305' 'CA-317' 'CA-510' 'CA-315' 'CA-321' 'CA-490'
 'CA-497' 'CA-489' 'CA-319' 'CA-492' 'CA-494' 'CA-502' 'CA-500' 'CA-495'
 'CA-498' 'CA-304' 'CA-307' 'CA-306' 'CA-501' 'CA-318' 'CA-308' 'CA-316'
 'CA-312' 'CA-320' 'CA-212' 'CA-508' 'CA-322' 'CA-314' 'CA-503' 'CA-504'
 'CA-496' 'CA-310' 'CA-506' 

In [11]:
# Very weird occurrence for Space ID 11900388, it doesn't look like all the others.
print("Amount of charging sessions with Space ID 11900388:", len(df_charging_sessions[df_charging_sessions["spaceID"] == "11900388"]))
# Visual look shows that there are no done charging time entries for this particular ID. So let's check this assumption:
print("Amount of charging sessions with Space ID 11900388 and no done charging time:", len(df_charging_sessions[(df_charging_sessions["spaceID"] == "11900388") & (df_charging_sessions["doneChargingTime"].isnull().values == True)]))

Amount of charging sessions with Space ID 11900388: 1125
Amount of charging sessions with Space ID 11900388 and no done charging time: 1125


In [12]:
# Something is wrong about this space. Let's delete all rows associated with it.
df_charging_sessions.drop(df_charging_sessions.index[df_charging_sessions['spaceID'] == "11900388"], inplace = True)

In [13]:
# To handle null values among the User ID, let's check the following statements:
print("There is a user ID that equals \"0\":", 0 in df_charging_sessions.userID.unique())
print("There are null values for the User ID in the dataset:", True in df_charging_sessions.userID.isnull().values)

There is a user ID that equals "0": False
There are null values for the User ID in the dataset: True


In [14]:
# Let's replace all null values in User ID with the User ID 0, so that the anonymous user has ID=0 and check previous statement again:
df_charging_sessions["userID"].loc[df_charging_sessions.userID.isnull()] = 0.0
print("There is a user ID that equals \"0\":", 0 in df_charging_sessions.userID.unique())
print("There are null values for the User ID in the dataset:", True in df_charging_sessions.userID.isnull().values)

There is a user ID that equals "0": True
There are null values for the User ID in the dataset: False


In [15]:
# Let's have a look on how the User Input looks like:
print(df_charging_sessions["userInputs"].head(1).values)

["[{'WhPerMile': 250, 'kWhRequested': 25.0, 'milesRequested': 100, 'minutesAvailable': 463, 'modifiedAt': 'Thu, 02 Jan 2020 13:09:39 GMT', 'paymentRequired': True, 'requestedDeparture': 'Thu, 02 Jan 2020 20:51:54 GMT', 'userID': 194}]"]


In [16]:
# To handle null values in User Input for anonymous users, let's replace those cells with empty lists:
df_charging_sessions["userInputs"].loc[df_charging_sessions.userInputs.isnull()] = "[]"

In [17]:
# To better handle timestamps, let's bring them into a suitable format and convert them to Los Angeles timezone:
df_charging_sessions['connectionTime'] = pd.to_datetime(df_charging_sessions['connectionTime']).dt.tz_convert('America/Los_Angeles')
df_charging_sessions['disconnectTime'] = pd.to_datetime(df_charging_sessions['disconnectTime']).dt.tz_convert('America/Los_Angeles')
df_charging_sessions['doneChargingTime'] = pd.to_datetime(df_charging_sessions['doneChargingTime']).dt.tz_convert('America/Los_Angeles')

In [18]:
# Where there is no finished charging time, let's assume charging finished when vehicle was plugged out:
df_charging_sessions["doneChargingTime"].loc[
df_charging_sessions.doneChargingTime.isnull()] = df_charging_sessions["disconnectTime"].loc[df_charging_sessions.doneChargingTime.isnull()]

In [19]:
# Let's check the values of the overall time vehicles were connected:
time_connected = (df_charging_sessions.disconnectTime - df_charging_sessions.connectionTime).dt.total_seconds()
print("Minimum time connected:", time_connected.min(),"\bs")
print("Maximum time connected:", time_connected.max(),"\bs")
print("Average time connected:", time_connected.mean(),"\bs")

Minimum time connected: 183.0s
Maximum time connected: 882969.0s
Average time connected: 22936.419389160095s


In [20]:
# Check how idle time looks like. This is the time where the vehicle was still plugged in but done charging:
time_idle = (df_charging_sessions.disconnectTime - df_charging_sessions.doneChargingTime).dt.total_seconds()
print("Minimum idle time:", time_idle.min(),"\bs")
print("Maximum idle time:", time_idle.max(),"\bs")
print("Average idle time:", time_idle.mean(),"\bs")

Minimum idle time: -3596.0s
Maximum idle time: 562037.0s
Average idle time: 9795.857945299787s


In [21]:
# How many are there having negative values??
print("Negative idle times:", (time_idle < 0).sum())

Negative idle times: 4387


In [22]:
# Maybe there was a sync error because viusally we can see that most of the times done charging time is set ~2 minutes after plug out.
# This means that a lot of those values may be set to disconnect time
df_charging_sessions["doneChargingTime"].loc[((df_charging_sessions.disconnectTime - df_charging_sessions.doneChargingTime).dt.total_seconds() > -120) &
((df_charging_sessions.disconnectTime - df_charging_sessions.doneChargingTime).dt.total_seconds() < 0)] = df_charging_sessions["disconnectTime"].loc[
((df_charging_sessions.disconnectTime - df_charging_sessions.doneChargingTime).dt.total_seconds() > -120) &
((df_charging_sessions.disconnectTime - df_charging_sessions.doneChargingTime).dt.total_seconds() < 0)]

In [23]:
# Now let's check again:
time_idle = (df_charging_sessions.disconnectTime - df_charging_sessions.doneChargingTime).dt.total_seconds()
print("Negative idle times:", (time_idle < 0).sum())

Negative idle times: 2


In [24]:
# Drop the two remaining rows that seem to be erroneous:
df_charging_sessions.drop(df_charging_sessions.index[
((df_charging_sessions.disconnectTime - df_charging_sessions.doneChargingTime).dt.total_seconds() < -120)], inplace = True)

In [25]:
# Now let's check again:
time_idle = (df_charging_sessions.disconnectTime - df_charging_sessions.doneChargingTime).dt.total_seconds()
print("Negative idle times:", (time_idle < 0).sum())

Negative idle times: 0


In [26]:
# Have a look on total time range information:
print("Observation start date:", df_charging_sessions.connectionTime.min())
print("Observation end date:", df_charging_sessions.disconnectTime.max())
print("Total observation interval:", df_charging_sessions.disconnectTime.max() - df_charging_sessions.connectionTime.min())

Observation start date: 2018-04-25 04:08:04-07:00
Observation end date: 2021-09-14 07:46:28-07:00
Total observation interval: 1238 days 03:38:24


In [27]:
# Now it is necessary to check the actual charging times for errors with same procedure as above:
time_charging = (df_charging_sessions.doneChargingTime - df_charging_sessions.connectionTime).dt.total_seconds()
print("Minimum charging time:", time_charging.min(),"\bs")
print("Maximum charging time:", time_charging.max(),"\bs")
print("Average charging time:", time_charging.mean(),"\bs")

Minimum charging time: -2482.0s
Maximum charging time: 720057.0s
Average charging time: 13136.965560945078s


In [28]:
# Done charging before even plugged in? Interesting isn't it...
print("Amount of magic sessions where charging was done before even plugged in:", (time_charging < 0).sum())

Amount of magic sessions where charging was done before even plugged in: 27


In [29]:
# The 27 magic sessions shall vanish...
df_charging_sessions.drop(df_charging_sessions.index[
((df_charging_sessions.doneChargingTime - df_charging_sessions.connectionTime).dt.total_seconds() < 0)], inplace = True)

In [30]:
# Just to get sure...
time_charging = (df_charging_sessions.doneChargingTime - df_charging_sessions.connectionTime).dt.total_seconds()
print("Amount of magic sessions where charging was done before even plugged in:", (time_charging < 0).sum())

Amount of magic sessions where charging was done before even plugged in: 0


In [31]:
# We have to format the user inputs to check them for null values easier:
userInputs = df_charging_sessions.userInputs.tolist()
for i in range(0, len(userInputs)):
    userInputs[i] = json.loads(userInputs[i].replace("'", "\"").replace(": False",": false").replace(": True",": true"))

df_charging_sessions.userInputs = userInputs

In [32]:
# Also reset the index for iteration purposes:
df_charging_sessions = df_charging_sessions.reset_index(drop=True)

In [33]:
# This script will check for any missing values across all user input data:
c = 0
for i in range(0, len(df_charging_sessions.userInputs)):
    if not len(df_charging_sessions.userInputs[i]) == 0:
        for j in range(0, len(df_charging_sessions.userInputs[i])):
            for item in df_charging_sessions.userInputs[i][j].values():
                if item == None or item == "NaN" or item == "":
                    c+=1
                    
print("Missing values in User Input:", c)

Missing values in User Input: 0


In [34]:
# Final look on column info:
df_charging_sessions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63883 entries, 0 to 63882
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype                              
---  ------            --------------  -----                              
 0   id                63883 non-null  object                             
 1   connectionTime    63883 non-null  datetime64[ns, America/Los_Angeles]
 2   disconnectTime    63883 non-null  datetime64[ns, America/Los_Angeles]
 3   doneChargingTime  63883 non-null  datetime64[ns, America/Los_Angeles]
 4   kWhDelivered      63883 non-null  float64                            
 5   sessionID         63883 non-null  object                             
 6   siteID            63883 non-null  int64                              
 7   spaceID           63883 non-null  object                             
 8   stationID         63883 non-null  object                             
 9   userID            63883 non-null  float64                    

In [35]:
# Dataset cleaned, sort everything by date and time and export the resulting dataset:
df_charging_sessions.sort_values('connectionTime', inplace=True)
df_charging_sessions.to_csv("../datasets/charging_sessions_cleaned.csv", index=False)

#### 1.2 Cleaning Weather Data

In [36]:
# Load weather data and have first look on it:
df_weather = pd.DataFrame(pd.read_csv("../datasets/weather_burbank_airport.csv"))
df_weather.head(5)

,city,timestamp,temperature,cloud_cover,cloud_cover_description,pressure,windspeed,precipitation,felt_temperature
0,Burbank,2018-01-01 08:53:00,9.0,33.0,Fair,991.75,9.0,0.0,8.0
1,Burbank,2018-01-01 09:53:00,9.0,33.0,Fair,992.08,0.0,0.0,9.0
2,Burbank,2018-01-01 10:53:00,9.0,21.0,Haze,992.08,0.0,0.0,9.0
3,Burbank,2018-01-01 11:53:00,9.0,29.0,Partly Cloudy,992.08,0.0,0.0,9.0
4,Burbank,2018-01-01 12:53:00,8.0,33.0,Fair,992.08,0.0,0.0,8.0


In [37]:
# Show information about all columns:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29244 entries, 0 to 29243
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   city                     29244 non-null  object 
 1   timestamp                29244 non-null  object 
 2   temperature              29219 non-null  float64
 3   cloud_cover              29224 non-null  float64
 4   cloud_cover_description  29224 non-null  object 
 5   pressure                 29236 non-null  float64
 6   windspeed                29158 non-null  float64
 7   precipitation            29244 non-null  float64
 8   felt_temperature         29218 non-null  float64
dtypes: float64(6), object(3)
memory usage: 2.0+ MB


In [38]:
# We have to drop the rows with null values because we cannot simply replace them:
df_weather.drop(df_weather.index[df_weather['temperature'].isna()], inplace = True)
df_weather.drop(df_weather.index[df_weather['cloud_cover_description'].isna()], inplace = True)
df_weather.drop(df_weather.index[df_weather['pressure'].isna()], inplace = True)
df_weather.drop(df_weather.index[df_weather['windspeed'].isna()], inplace = True)

In [39]:
# Drop duplicates if present:
df_weather = df_weather.drop_duplicates(keep="first")

In [40]:
# Let's see how many rows we lost:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29105 entries, 0 to 29243
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   city                     29105 non-null  object 
 1   timestamp                29105 non-null  object 
 2   temperature              29105 non-null  float64
 3   cloud_cover              29105 non-null  float64
 4   cloud_cover_description  29105 non-null  object 
 5   pressure                 29105 non-null  float64
 6   windspeed                29105 non-null  float64
 7   precipitation            29105 non-null  float64
 8   felt_temperature         29105 non-null  float64
dtypes: float64(6), object(3)
memory usage: 2.2+ MB


In [41]:
# Check unique String values:
print("Unique cities:", df_weather.city.unique())
print("Unique weather description:", df_weather.cloud_cover_description.unique())

Unique cities: ['Burbank']
Unique weather description: ['Fair' 'Haze' 'Partly Cloudy' 'Mostly Cloudy' 'Cloudy' 'Fog' 'Light Rain'
 'Rain' 'Heavy Rain' 'Heavy Rain / Windy' 'Light Rain / Windy' 'T-Storm'
 'Fair / Windy' 'Cloudy / Windy' 'Mostly Cloudy / Windy'
 'Partly Cloudy / Windy' 'Thunder in the Vicinity' 'Thunder' 'Smoke'
 'Light Rain with Thunder' 'Heavy T-Storm' 'Rain / Windy' 'Blowing Dust']


In [42]:
# Adjust timestamp so that it has an identical format just like the other dataset:
df_weather['timestamp'] = pd.to_datetime(df_weather['timestamp']).dt.tz_localize(tz='America/Los_Angeles', ambiguous='NaT', nonexistent='shift_forward')

In [43]:
# Delete rows with an erroneous timestamp (there are only three such rows):
print("Amount of rows with corrupt timestamps:", len(df_weather[df_weather.timestamp.isna() == True]))
df_weather = df_weather[df_weather.timestamp.isna() == False]

Amount of rows with corrupt timestamps: 3


In [44]:
# Now let's check all measurements for feasibility:
scales = ["temperature", "cloud cover", "pressure", "windspeed", "precipitation", "felt temperature"]
unit = ["\b°C", "\b%", "mBar", "km/h", "mm", "\b°C"]
scales_in_df = list(df_weather.columns.values)
scales_in_df.pop(0)
scales_in_df.pop(0)
scales_in_df.pop(2)
i = 0
for scale in scales_in_df:
    print(f"Minimum measured {scales[i]}:", df_weather[scale].min(), unit[i])
    print(f"Maximum measured {scales[i]}:", df_weather[scale].max(), unit[i])
    print(f"Average measured {scales[i]}:", df_weather[scale].mean(), unit[i])
    print()
    i+=1


Minimum measured temperature: 2.0°C
Maximum measured temperature: 46.0°C
Average measured temperature: 17.873410762146932°C

Minimum measured cloud cover: 4.0%
Maximum measured cloud cover: 47.0%
Average measured cloud cover: 30.108068174008658%

Minimum measured pressure: 971.0 mBar
Maximum measured pressure: 999.65 mBar
Average measured pressure: 986.879853961927 mBar

Minimum measured windspeed: 0.0 km/h
Maximum measured windspeed: 57.0 km/h
Average measured windspeed: 8.57226307470277 km/h

Minimum measured precipitation: 0.0 mm
Maximum measured precipitation: 18.54 mm
Average measured precipitation: 0.05959727853755756 mm

Minimum measured felt temperature: 0.0°C
Maximum measured felt temperature: 42.0°C
Average measured felt temperature: 17.71881657618033°C



In [45]:
# We don't need the city column, felt temperature, and cloud cover:
df_weather = df_weather.drop(["city", "felt_temperature", "cloud_cover"], axis=1)
# Cloud cover description can be renamed to weather:
df_weather.rename(columns={"cloud_cover_description":"weather"}, inplace=True)

In [46]:
# Show information about all columns for a last time:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29102 entries, 0 to 29243
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype                              
---  ------         --------------  -----                              
 0   timestamp      29102 non-null  datetime64[ns, America/Los_Angeles]
 1   temperature    29102 non-null  float64                            
 2   weather        29102 non-null  object                             
 3   pressure       29102 non-null  float64                            
 4   windspeed      29102 non-null  float64                            
 5   precipitation  29102 non-null  float64                            
dtypes: datetime64[ns, America/Los_Angeles](1), float64(4), object(1)
memory usage: 1.6+ MB


In [47]:
# Weather data is now cleaned, export it ordered by date and time:
df_weather.sort_values('timestamp', inplace=True)
df_weather.to_csv("../datasets/weather_burbank_airport_cleaned.csv", index=False)

#### 1.3 Cleaning Weather for 2021 (Extension Data)

In [48]:
# This dataset was downloaded from https://www.weather.gov/wrh/timeseries?site=KBUR and processed with help of https://www.convertcsv.com/html-table-to-csv.htm
df_2021 = pd.DataFrame(pd.read_csv("../datasets/2021.csv"))

In [49]:
# Get overview of data
df_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8940 entries, 0 to 8939
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Date/Time(L)             8940 non-null   object 
 1   Temp. (°C)               8938 non-null   float64
 2   Dew Point (°C)           8937 non-null   float64
 3   Relative Humidity (%)    8937 non-null   float64
 4   Heat Index (°C)          1086 non-null   float64
 5   Wind Chill (°C)          33 non-null     float64
 6   Wind Direction           8536 non-null   object 
 7   Wind Speed (kph)         8940 non-null   object 
 8   Visibility (km)          8939 non-null   float64
 9   Weather                  912 non-null    object 
 10  Clouds (x100 ft)         8924 non-null   object 
 11  Sea Level Pressure (mb)  8684 non-null   float64
 12  Station Pressure (mb)    8940 non-null   float64
 13  Altimeter Setting (mb)   8940 non-null   float64
 14  1 Hour Precip (mm)      

In [50]:
# Drop unnecessary columns and rename to match the columns of given weather dataset
df_2021.drop(["Dew Point (°C)", "Relative Humidity (%)", "Heat Index (°C)", "Wind Chill (°C)", "Wind Direction", "Visibility (km)", "Clouds (x100 ft)", "Sea Level Pressure (mb)", "Altimeter Setting (mb)", "3 Hour Precip (mm)", "6 Hour Precip (mm)", "24 Hour Precip (mm)", "6 Hr Max (°C)", "6 Hr Min (°C)", "24 Hr Max (°C)", "24 Hr Min (°C)"], axis=1, inplace=True)
df_2021.rename(columns={"Temp. (°C)": "temperature", "Date/Time(L)":"timestamp", "Wind Speed (kph)":"windspeed", "Weather":"weather", "Station Pressure (mb)":"pressure", "1 Hour Precip (mm)":"precipitation"}, inplace=True)

In [51]:
# Check how data columns look like:
df_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8940 entries, 0 to 8939
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   timestamp      8940 non-null   object 
 1   temperature    8938 non-null   float64
 2   windspeed      8940 non-null   object 
 3   weather        912 non-null    object 
 4   pressure       8940 non-null   float64
 5   precipitation  340 non-null    float64
dtypes: float64(3), object(3)
memory usage: 419.2+ KB


In [52]:
df_2021.drop(df_2021.index[df_2021['temperature'].isna()], inplace = True)

In [53]:
# This cell suppresses copy warnings generated by python.
pd.options.mode.chained_assignment = None

In [54]:
# Fill all cells that have no weather information with "Fair"
df_2021.weather.loc[df_2021["weather"].isna()] = "Fair"

In [55]:
# Check unique weather descriptions:
df_2021.weather.unique()

array(['Fair', 'Lt rain', 'Lt rain, Mist', 'Hvy rain', 'Rain, Mist',
       'Rain', 'Hvy rain, Mist', 'Haze', 'Mist', 'Fog',
       'Thunder, Lt rain', 'Lt thunder shwr', 'Smoke', 'Thunder'],
      dtype=object)

In [56]:
# Check unique windspeed values:
df_2021.windspeed.unique()

array(['9', '15', '13', '17', '11', '6', '7', '0', '19', '20', '17G35',
       '19G37', '24G39', '20G41', '22', '26G37', '26', '22G33', '19G31',
       '28', '22G30', '13G37', '35G56', '26G41', '30G41', '28G41',
       '26G33', '24G33', '24', '20G43', '22G35', '41G54', '20G48',
       '24G35', '28G35', '30G43', '31G41', '17G31', '15G28', '17G30',
       '19G41', '35G69', '46G59', '54G74', '65G78', '52G87', '50G74',
       '33G63', '20G37', '22G37', '13G28', '13G35', '24G31', '15G37',
       '20G30', '22G39', '20G33', '22G48', '28G46', '30G54', '35G48',
       '28G48', '28G43', '28G44', '44G61', '24G37', '19G44', '43G61',
       '35G52', '30', '15G30', '17G28', '20G28', '19G28', '13G30',
       '20G31', '19G26', '13G26', '19G33', '17G26', '11G30', '19G30',
       '22G31', '30G44', '30G39', '26G35', '15G31', '15G26', '28G39',
       '11G31', '30G48', '22G44', '28G37', '30G50', '26G54', '17G33',
       '6G31', '20G26', '24G30', '26G44', '17G37', '37G59', '41G57',
       '43G52', '33G46', 

In [57]:
# Fill missing precipitation data with zeroes:
df_2021.precipitation = df_2021.precipitation.fillna(0)

In [58]:
# The windspeed is a String column that sometimes contains a "Gxx" suffix. This one will be cut off by the following code:
df_2021["windspeed"].loc[df_2021.windspeed.str.len() == 5] = df_2021["windspeed"].loc[df_2021.windspeed.str.len() == 5].str[0:1]
df_2021["windspeed"].loc[df_2021.windspeed.str.len() == 4] = df_2021["windspeed"].loc[df_2021.windspeed.str.len() == 4].str[0]
df_2021.windspeed = df_2021.windspeed.astype("float64")

In [59]:
# See how many entries have a wrong weather condition assigned:
df_2021.loc[(df_2021.weather == "Fair") & (df_2021.precipitation > 0.)]

,timestamp,temperature,windspeed,weather,pressure,precipitation
21,"Dec 30, 6:53 pm",12.0,13.0,Fair,982.86,0.03
56,"Dec 29, 10:53 am",11.0,17.0,Fair,986.16,0.25
100,"Dec 27, 5:53 pm",10.0,9.0,Fair,984.84,0.03
141,"Dec 26, 1:53 am",11.0,6.0,Fair,988.14,0.03
142,"Dec 26, 12:53 am",11.0,9.0,Fair,987.48,0.51
...,...,...,...,...,...,...
8403,"Jan 23, 3:53 am",8.0,0.0,Fair,986.16,0.25
8405,"Jan 23, 2:53 am",9.0,7.0,Fair,986.82,0.03
8488,"Jan 19, 8:53 pm",17.0,3.0,Fair,985.83,0.03
8515,"Jan 18, 6:53 pm",20.0,9.0,Fair,980.88,0.03


In [60]:
# Correct weather assignment accordingly:
df_2021.weather.loc[(df_2021.weather == "Fair") & (df_2021.precipitation > 0.) & (df_2021.precipitation < 1.)] = "Light Rain"
df_2021.weather.loc[(df_2021.weather == "Fair") & (df_2021.precipitation >= 1.) & (df_2021.precipitation < 2.)] = "Rain"
df_2021.weather.loc[(df_2021.weather == "Fair") & (df_2021.precipitation >= 2.)] = "Heavy Rain"

In [61]:
# Check unique precipitation values:
df_2021.precipitation.unique()

array([ 0.  ,  0.03,  1.52,  0.25,  1.27,  0.76,  1.02,  2.03,  3.3 ,
        3.05,  5.33,  4.57,  5.08,  4.32,  4.06,  3.56,  2.54,  0.51,
        4.83,  2.29,  3.81, 14.99,  6.1 ,  6.86,  6.6 ,  5.84,  7.11,
        1.78,  5.59,  2.79,  9.14,  7.87,  9.65])

In [62]:
# Load original weather data and check unique weather names:
df_weather_cleaned = pd.DataFrame(pd.read_csv("../datasets/weather_burbank_airport_cleaned.csv"))
df_weather_cleaned.weather.unique()

array(['Fair', 'Haze', 'Partly Cloudy', 'Mostly Cloudy', 'Cloudy', 'Fog',
       'Light Rain', 'Rain', 'Heavy Rain', 'Heavy Rain / Windy',
       'Light Rain / Windy', 'T-Storm', 'Fair / Windy', 'Cloudy / Windy',
       'Mostly Cloudy / Windy', 'Partly Cloudy / Windy',
       'Thunder in the Vicinity', 'Thunder', 'Smoke',
       'Light Rain with Thunder', 'Heavy T-Storm', 'Rain / Windy',
       'Blowing Dust'], dtype=object)

In [63]:
# Rename some of the weather names to match the names from existing weather data:
df_2021.weather.loc[df_2021.weather == "Lt rain"] = "Light Rain"
df_2021.weather.loc[df_2021.weather == "Lt rain, Mist"] = "Light Rain / Mist"
df_2021.weather.loc[df_2021.weather == "Hvy rain"] = "Heavy Rain"
df_2021.weather.loc[df_2021.weather == "Rain, Mist"] = "Rain / Mist"
df_2021.weather.loc[df_2021.weather == "Hvy rain, Mist"] = "Heavy Rain / Mist"
df_2021.weather.loc[df_2021.weather == "Thunder, Lt rain"] = "Light Rain with Thunder"
df_2021.weather.loc[df_2021.weather == "Lt thunder shwr"] = "Heavy Rain with Light Thunder"

In [64]:
# Check unique temperature values:
df_2021.temperature.unique()

array([14., 16., 15., 13., 12., 11.,  9.,  8., 10.,  7.,  5.,  4.,  6.,
       17., 18., 21., 22., 20., 19.,  3., 24., 26., 28., 30., 31., 29.,
       23., 27., 25., 33., 32., 34., 36., 35., 37., 38., 39., 40.])

In [65]:
# Check unique pressure values:
df_2021.pressure.unique()

array([ 979.23,  978.57,  978.9 ,  979.89,  980.88,  981.54,  981.21,
        980.55,  980.22,  982.53,  983.19,  982.86,  983.52,  983.85,
        984.51,  984.84,  984.18,  985.17,  986.16,  985.5 ,  985.83,
        986.82,  987.81,  988.47,  989.46,  990.45,  991.44,  991.11,
        991.77,  992.1 ,  990.78,  990.12,  992.76,  992.43,  989.13,
        988.8 ,  988.14,  987.48,  987.15,  989.79,  986.49,  982.2 ,
        993.75,  993.09,  993.42,  994.08,  994.41,  994.74,  995.39,
        995.72,  995.06,  996.05,  996.38,  997.37,  997.7 ,  997.04,
        996.71,  998.36,  998.03,  981.87,  977.25,  976.59,  977.91,
        979.56,  978.24,  999.02,  999.68, 1000.01,  999.35,  998.69])

In [66]:
# The following script converts unrecognizable format of date and times to the format used by pandas timedate object.
# We assume that times in the retreived data are already local, which plays later an important role.
a = list(df_2021['timestamp'])
months = {"Jan":"01","Feb":"02","Mar":"03","Apr":"04","May":"05","Jun":"06","Jul":"07","Aug":"08","Sep":"09","Oct":"10","Nov":"11","Dec":"12"}
for i in range(0, len(a)):
    b = a[i].split(" ")
    c = "2021-"
    if len(b[1]) == 3:
        c = c + months.get(b[0]) + "-" + b[1][0:2] + " "
    else:
        c = c + months.get(b[0]) + "-0" + b[1][0:1] + " "
    if b[3] == "pm" and (int(b[2].split(":")[0])) < 12:
        b[2] = str(int(b[2].split(":")[0]) + 12) + ":" + b[2].split(":")[1]
    if len(b[2]) == 5:
        c = c + b[2] + ":00"
    else:
        c = c + "0" + b[2] + ":00"
    a[i] = c
df_2021.timestamp = a

In [67]:
# Check the data and especially new timestamp format:
df_2021.head()

,timestamp,temperature,windspeed,weather,pressure,precipitation
0,2021-12-31 15:53:00,14.0,9.0,Fair,979.23,0.0
1,2021-12-31 14:53:00,14.0,15.0,Fair,978.57,0.0
2,2021-12-31 13:53:00,16.0,13.0,Fair,978.90,0.0
3,2021-12-31 12:53:00,16.0,17.0,Fair,978.90,0.0
4,2021-12-31 11:53:00,16.0,11.0,Fair,979.89,0.0


In [68]:
# Convert timestamps to LA timezone. Some times will be flagged as nonexistent and will be deleted in next step. There are only 1-2 times with such error.
df_2021['timestamp'] = pd.to_datetime(df_2021['timestamp']).dt.tz_localize(tz='America/Los_Angeles', ambiguous='NaT', nonexistent='shift_forward')

In [69]:
# Clear NaT rows:
df_2021 = df_2021[df_2021.timestamp.isna() == False]

In [70]:
# Rearrange columns to get same order as in original weather dataset:
df_2021 = df_2021.reindex(columns=['timestamp', 'temperature', 'weather', 'pressure', 'windspeed', 'precipitation'])

In [71]:
# Weather data is now cleaned, export it ordered by date and time:
df_2021.sort_values('timestamp', inplace=True)
df_2021.to_csv("../datasets/2021_cleaned.csv", index=False)

#### 1.4 Extending Weather Data

In [72]:
# Remove all data for 2021 that is included in original dataset:
df_weather = df_weather[df_weather.timestamp.dt.year != 2021]

In [73]:
# Import cleaned weather for 2021:
df_2021_cleaned = pd.DataFrame(pd.read_csv("../datasets/2021_cleaned.csv"))

In [74]:
# Stack data for 2021 on original dataset:
df_weather_extended = pd.concat([df_weather, df_2021_cleaned], ignore_index=True)

In [75]:
# Check how the dataset structure looks like and how many rows we achieved:
df_weather_extended.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38030 entries, 0 to 38029
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   timestamp      38030 non-null  object 
 1   temperature    38030 non-null  float64
 2   weather        38030 non-null  object 
 3   pressure       38030 non-null  float64
 4   windspeed      38030 non-null  float64
 5   precipitation  38030 non-null  float64
dtypes: float64(4), object(2)
memory usage: 1.7+ MB


In [76]:
# Convert timestamp to datetime object:
df_weather_extended['timestamp'] = pd.to_datetime(df_weather_extended['timestamp'], utc=True).dt.tz_convert('America/Los_Angeles')

In [77]:
# Export extended weather data as CSV:
df_weather_extended.to_csv("../datasets/df_weather_cleaned_extended.csv", index=False)

#### 1.5 Merging Weather Data on Charging Sessions

In [78]:
# For merging we have to rename our timestamp column:
df_weather_extended.rename(columns={"timestamp": "connectionTime"}, inplace=True)
df_weather_extended.sort_values("connectionTime", inplace=True)

In [79]:
df_weather_extended.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38030 entries, 0 to 38029
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype                              
---  ------          --------------  -----                              
 0   connectionTime  38030 non-null  datetime64[ns, America/Los_Angeles]
 1   temperature     38030 non-null  float64                            
 2   weather         38030 non-null  object                             
 3   pressure        38030 non-null  float64                            
 4   windspeed       38030 non-null  float64                            
 5   precipitation   38030 non-null  float64                            
dtypes: datetime64[ns, America/Los_Angeles](1), float64(4), object(1)
memory usage: 2.0+ MB


In [80]:
# Now we can merge with a single-line magic command based on timestamps.
# This means we perform a left-join and assign closest weather entries to the session start times.
df_charging_sessions_with_weather = pd.merge_asof(df_charging_sessions, df_weather_extended, on="connectionTime", direction="nearest")

In [81]:
# Let's check the columns:
df_charging_sessions_with_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63883 entries, 0 to 63882
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype                              
---  ------            --------------  -----                              
 0   id                63883 non-null  object                             
 1   connectionTime    63883 non-null  datetime64[ns, America/Los_Angeles]
 2   disconnectTime    63883 non-null  datetime64[ns, America/Los_Angeles]
 3   doneChargingTime  63883 non-null  datetime64[ns, America/Los_Angeles]
 4   kWhDelivered      63883 non-null  float64                            
 5   sessionID         63883 non-null  object                             
 6   siteID            63883 non-null  int64                              
 7   spaceID           63883 non-null  object                             
 8   stationID         63883 non-null  object                             
 9   userID            63883 non-null  float64                    

In [82]:
# Also look at a few data entries:
df_charging_sessions_with_weather.head(5)

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,userID,userInputs,temperature,weather,pressure,windspeed,precipitation
0,5bc90cb9f9af8b0d7fe77cd2,2018-04-25 04:08:04-07:00,2018-04-25 06:20:10-07:00,2018-04-25 06:20:10-07:00,7.932,2_39_78_362_2018-04-25 11:08:04.400812,2,CA-496,2-39-78-362,0.0,[],18.0,Partly Cloudy,987.47,7.0,0.0
1,5bc90cb9f9af8b0d7fe77cd3,2018-04-25 06:45:10-07:00,2018-04-25 17:56:16-07:00,2018-04-25 09:44:15-07:00,10.013,2_39_95_27_2018-04-25 13:45:09.617470,2,CA-319,2-39-95-27,0.0,[],16.0,Fair,988.78,13.0,0.0
2,5bc90cb9f9af8b0d7fe77cd4,2018-04-25 06:45:50-07:00,2018-04-25 16:04:45-07:00,2018-04-25 07:51:44-07:00,5.257,2_39_79_380_2018-04-25 13:45:49.962001,2,CA-489,2-39-79-380,0.0,[],16.0,Fair,988.78,13.0,0.0
3,5bc90cb9f9af8b0d7fe77cd5,2018-04-25 07:37:06-07:00,2018-04-25 16:55:34-07:00,2018-04-25 09:05:22-07:00,5.177,2_39_79_379_2018-04-25 14:37:06.460772,2,CA-327,2-39-79-379,0.0,[],15.0,Fair,988.78,13.0,0.0
4,5bc90cb9f9af8b0d7fe77cd6,2018-04-25 07:40:34-07:00,2018-04-25 16:03:12-07:00,2018-04-25 10:40:30-07:00,10.119,2_39_79_381_2018-04-25 14:40:33.638896,2,CA-490,2-39-79-381,0.0,[],15.0,Fair,988.78,13.0,0.0


In [83]:
# Export the new dataset:
df_charging_sessions_with_weather.to_csv("../datasets/df_charging_sessions_with_weather.csv", index=False)